# <center>Machine Learning Project</center>

** **
## <center>*03.7 - Neural Networks*</center>

** **

The members of the `team` are:
- Ana Farinha - 20211514
- Francisco Capontes - 20211692
- Sofia Gomes - 20240848
- Rui Lourenço - 2021639



In [5]:
# Import libraries
import pandas as pd
import numpy as np

#make the split here
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.neural_network import MLPClassifier
from catboost import CatBoostClassifier

from sklearn.model_selection import StratifiedKFold
import time

from utils import *
from utils_feature_selection import check_performace
from utils_dicts import *

import warnings
warnings.filterwarnings('ignore')

## <span style="color:salmon"> 1. Import Dataset </span> 

In [6]:
# Import dataset
train_df = pd.read_csv('preprocessed_data/train_data.csv', index_col="Claim Identifier")

In [7]:
# Import dataset
test_df = pd.read_csv('./preprocessed_data/test_data.csv', index_col = 'Claim Identifier')

## <span style="color:salmon"> 2. Prepare Dataset </span> 

Define y as a target "Claim Injury Type Encoded" and X with all the other columns

In [13]:
 for col in numerical_features+categorical_features:
    if col not in train_df.columns:
        print(col)

Enc County of Injury
Enc District Name
Enc Industry Code
Enc WCIO Cause of Injury Code
Enc WCIO Nature of Injury Code
Enc WCIO Part Of Body Code
Enc Zip Code
Relative_Wage
Financial Impact Category
Age_Group


In [4]:
X = df.drop(["Claim Injury Type Encoded"], axis = 1)
y = df["Claim Injury Type Encoded"]

KeyError: "['Claim Injury Type Encoded'] not found in axis"

In [ ]:
drop_list = []
drop_list_2 = []

In [ ]:
for col in X_train.columns:
    if col not in essential_features:
        drop_list.append(col)
    if col not in reduced_featured:
        drop_list_2.append(col)

In [ ]:
X_essential = X.drop(drop_list, axis=1)
X_reduced = X.drop(drop_list_2, axis=1)

## <span style="color:salmon"> 3. CatBoosted </span> 

In [ ]:
config = {'iterations': 500, 'learning_rate': 0.15, 'depth': 6, 'l2_leaf_reg': 9.5}

In [ ]:
model = CatBoostClassifier(
        iterations=config["iterations"],
        learning_rate=config["learning_rate"],
        depth=config["depth"],
        l2_leaf_reg=config["l2_leaf_reg"],
        loss_function="MultiClass", 
        eval_metric="MultiClass",  
        custom_metric=['F1'], 
        verbose=0
    )

In [ ]:
check_performace(model,X_essential,y,numerical_features,n_folds = 5)

In [ ]:
check_performace(model,X_reduced,y,numerical_features,n_folds = 5)

#### <span style="color:salmon"> 3.1  Evaluate the model </span> 


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_essential,y, test_size = 0.75, stratify = y, shuffle = True)

In [ ]:
X_train, X_val = apply_frequency_encoding(X_train, X_val)
NA_imputer(X_train,X_val)
create_new_features(X_train,X_val)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_train_pred = model_cb.predict(X_train)

In [ ]:
y_val_pred = model_cb.predict(X_val)

In [ ]:
class_mapping = {
    0:'1. CANCELLED', 
    1:'2. NON-COMP',
    2:'3. MED ONLY', 
    3:'4. TEMPORARY',
    4:'5. PPD SCH LOSS', 
    5:'6. PPD NSL', 
    6:'7. PTD', 
    7:'8. DEATH'
}

# Use the values from class_mapping as the target names
target_names = list(class_mapping.values())

Compute confusion matrix to evaluate the accuracy of a classification

In [ ]:
# Evaluate the model
print("Confusion Matrix:")
print(confusion_matrix(y_train, y_train_pred))
print("\nClassification Report:")
print(classification_report(y_train, y_train_pred, target_names=target_names))

In [ ]:
# Evaluate the model
print("Confusion Matrix:")
print(confusion_matrix(y_val, y_val_pred))
print("\nClassification Report:")
print(classification_report(y_val, y_val_pred, target_names=target_names))


## <span style="color:salmon"> 4. Test Predictions </span> 

Make validation predictions:

In [ ]:
X, X_test = apply_frequency_encoding(X, X_test)
NA_imputer(X, X_test)
create_new_features(X, X_test)

In [ ]:
# Make validation predictions
y_test_pred = model.predict(X_test)
y_test_pred = y_test_pred.ravel()

In [ ]:
y_test_final = np.array([class_mapping[i] for i in y_test_pred])

In [ ]:
test_id = model.index

In [ ]:
submission_df = pd.DataFrame({
    'Claim Identifier': test_id,
    'Claim Injury Type': y_test_final
})

In [ ]:
if False:
    version = version_control()
    submission_df.to_csv(f'./submissions/Group49_Version{version:02}.csv', index=False)